In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
NFA_df=pd.read_csv("NFA_2018.csv")
print(NFA_df.shape)
print(NFA_df.columns)
NFA_df.head(10)

(87020, 15)
Index(['country', 'ISO alpha-3 code', 'UN_region', 'UN_subregion', 'year',
       'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon', 'total', 'Percapita GDP (2010 USD)',
       'population'],
      dtype='object')


,country,ISO alpha-3 code,UN_region,UN_subregion,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population
0,Armenia,ARM,Asia,Western Asia,1992,BiocapPerCap,1.611286e-01,0.135023,0.083836,0.013718,0.033669,0.000000e+00,4.273741e-01,949.033,3449000
1,Armenia,ARM,Asia,Western Asia,1992,BiocapTotGHA,5.558130e+05,465763.337400,289190.662300,47320.224590,116139.598200,0.000000e+00,1.474227e+06,949.033,3449000
2,Armenia,ARM,Asia,Western Asia,1992,EFConsPerCap,3.909225e-01,0.189137,0.000001,0.004138,0.033669,1.112225e+00,1.730092e+00,949.033,3449000
3,Armenia,ARM,Asia,Western Asia,1992,EFConsTotGHA,1.348487e+06,652429.066600,4.327841,14272.803690,116139.598200,3.836620e+06,5.967954e+06,949.033,3449000
4,Armenia,ARM,Asia,Western Asia,1992,EFExportsPerCap,1.124910e-03,0.002283,0.000000,0.000438,0.000000,4.819043e-02,5.203676e-02,949.033,3449000
5,Armenia,ARM,Asia,Western Asia,1992,EFExportsTotGHA,3.880378e+03,7875.331688,0.000000,1512.195296,0.000000,1.662329e+05,1.795008e+05,949.033,3449000
6,Armenia,ARM,Asia,Western Asia,1992,EFImportsPerCap,2.309189e-01,0.056397,0.000001,0.003312,0.000000,8.791121e-02,3.785406e-01,949.033,3449000
7,Armenia,ARM,Asia,Western Asia,1992,EFImportsTotGHA,7.965547e+05,194541.060900,4.327841,11426.058040,0.000000,3.032497e+05,1.305776e+06,949.033,3449000
8,Armenia,ARM,Asia,Western Asia,1992,EFProdPerCap,1.611286e-01,0.135023,0.000000,0.001264,0.033669,1.072504e+00,1.403588e+00,949.033,3449000
9,Armenia,ARM,Asia,Western Asia,1992,EFProdTotGHA,5.558130e+05,465763.337400,0.000000,4358.940944,116139.598200,3.699604e+06,4.841678e+06,949.033,3449000


In [3]:
countries_df=NFA_df[["ISO alpha-3 code","country"]].drop_duplicates(keep="first")

# find missing values in "code" column
print(len(countries_df["ISO alpha-3 code"].unique()) == len(countries_df["country"].unique()))
drop_list=countries_df[countries_df["ISO alpha-3 code"].isna()]["country"]
print(drop_list)
countries_df=countries_df[~(countries_df["country"].isin(drop_list))]

False
23020      Ethiopia PDR
69780    Sudan (former)
85700             World
Name: country, dtype: object


In [4]:
NFA_2_df=NFA_df[~(NFA_df["country"].isin(drop_list))]
print(len(NFA_2_df["country"].unique()))


193


In [5]:
regions_df=NFA_2_df[['country','UN_region','UN_subregion']].drop_duplicates(keep="first").reset_index(drop=True)
regions_df.head()

,country,UN_region,UN_subregion
0,Armenia,Asia,Western Asia
1,Afghanistan,Asia,Southern Asia
2,Albania,Europe,Southern Europe
3,Algeria,Africa,Northern Africa
4,Angola,Africa,Middle Africa


In [6]:
population_df=NFA_2_df[["country","year","population"]].drop_duplicates(keep="first").reset_index(drop=True)
population_df.head()

,country,year,population
0,Armenia,1992,3449000
1,Armenia,1993,3370000
2,Armenia,1994,3290000
3,Armenia,1995,3223000
4,Armenia,1996,3173000


In [8]:
records_list_df=NFA_df["record"].drop_duplicates(keep="first")
records_list_df


0       BiocapPerCap
1       BiocapTotGHA
2       EFConsPerCap
3       EFConsTotGHA
4    EFExportsPerCap
5    EFExportsTotGHA
6    EFImportsPerCap
7    EFImportsTotGHA
8       EFProdPerCap
9       EFProdTotGHA
Name: record, dtype: object

In [9]:
e_footprint_df=NFA_2_df[["country","year",'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon']].loc[NFA_df["record"].isin(["BiocapTotGHA"," EFConsTotGHA","EFExportsTotGHA","EFImportsTotGHA","EFProdTotGHA"])]
e_footprint_df.head()

,country,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon
1,Armenia,1992,BiocapTotGHA,555812.97260,465763.337400,289190.662300,47320.224590,116139.5982,0.000000e+00
5,Armenia,1992,EFExportsTotGHA,3880.37791,7875.331688,0.000000,1512.195296,0.0000,1.662329e+05
7,Armenia,1992,EFImportsTotGHA,796554.71840,194541.060900,4.327841,11426.058040,0.0000,3.032497e+05
9,Armenia,1992,EFProdTotGHA,555812.97260,465763.337400,0.000000,4358.940944,116139.5982,3.699604e+06
11,Armenia,1993,BiocapTotGHA,538659.09590,466181.595600,289695.836700,47293.877270,107720.2388,0.000000e+00


## countries' land area from wikipedia page

In [10]:
url="https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area"
country_area_list=pd.read_html(url)
country_area_df=country_area_list[0]
columns=list(country_area_df.loc[0])
print(columns)
columns=['Rank', 'country', 'Total in km2 (mi2)', 'Land in km2 (mi2)', 'Water in km2 (mi2)', '% water', 'Notes']
country_area_df.columns=columns
country_area_df=country_area_df.iloc[2:]
country_area_df=country_area_df.drop(columns="Rank").reset_index(drop=True)
print(country_area_df.shape)
country_area_df.head()

['Rank', 'Sovereign state/dependency', 'Total in km2 (mi2)', 'Land in km2 (mi2)', 'Water in km2 (mi2)', '% water', 'Notes']
(261, 6)


,country,Total in km2 (mi2),Land in km2 (mi2),Water in km2 (mi2),% water,Notes
0,Russia,"(6,601,670)","(6,323,482)","(278,200)",4.21,Largest country in the world (10.995% of the w...
1,Antarctica,"(5,400,000)","(5,400,000)",(0),0,"13,720,000 km2 (5,300,000 sq mi) (98%) of land..."
2,Canada,"(3,855,100)","(3,511,023)","(344,080)",8.93,Largest English and French-speaking country an...
3,China,"(3,705,407)","(3,600,950)","(104,460)",2.82,Second largest country in Asia (though the lar...
4,United States,"(3,677,649)[7] –(3,796,742)","(3,531,905) –(3,531,925)[7]","(145,724)[7] –(264,837)",3.96–6.97,Second largest English-speaking country and th...


In [11]:
country_area_df=country_area_df[country_area_df["country"].isin(countries_df["country"])]
country_area_df.shape

(158, 6)

In [12]:
trial=countries_df
trial_new=pd.merge(trial, country_area_df,on="country",how="outer").sort_values("country")
len(trial_new["country"].unique())
trial_new

,ISO alpha-3 code,country,Total in km2 (mi2),Land in km2 (mi2),Water in km2 (mi2),% water,Notes
1,AFG,Afghanistan,"(251,830)","(251,830)",(0),0,NaN
2,ALB,Albania,"(11,100)","(10,578)",(520),4.7,NaN
3,DZA,Algeria,"(919,595)","(919,595)",(0),0,Largest Arabic-speaking and Berber-speaking co...
4,AGO,Angola,"(481,400)","(481,400)",(0),0,NaN
5,ATG,Antigua and Barbuda,(171),(170.9),(0),0,NaN
6,ARG,Argentina,"(1,073,500)","(1,056,640)","(16,880)",1.57,Largest Spanish-speaking country and second la...
0,ARM,Armenia,"(11,484)","(10,943)",(541),4.71,NaN
18,ABW,Aruba,NaN,NaN,NaN,NaN,NaN
7,AUS,Australia,"(2,969,907)","(2,947,336)","(22,571)",0.76,Third largest English-speaking country and lar...
8,AUT,Austria,"(32,383)","(31,832)",(551),1.7,NaN


In [13]:
rds_connection_string = "localhost:mohammad@127.0.0.1/co2_emissions"
engine = create_engine(f'mysql://{rds_connection_string}')